## Generate histograms

In [15]:
import zdb
import glob
import yaml

In [16]:
def generate_yaml(hists, selection, filepath, tables={"central": "Events"}, systs=[], mc=False):
    hists_def = {
        "_".join(k): [
            {"table": "df: '{table_name}'"},
            {"varname0": "df: '{}'".format(k[0])},
            {"varname1": "df: '{}'".format(k[1])},
            {"selection": "df: '{selection_name}'"},
            {"parent": "df: df.parent"},
            {"binvar0": "df: df.{}_bin".format(k[0])},
            {"binvar1": "df: df.{}_bin".format(k[1])},
            {"count": "df: 1."},
            {"sum_w": "df: {weight}"},
            {"sum_ww": "df: df.sum_w**2"},
            {"sum_wZCand_pt": "df: df.sum_w*df.ZCand_pt"},
        ] for k in hists
    }
    
    if mc:
        for key, hdef in hists_def.items():
            for label, vari in systs:
                hdef.extend([
                    {"sum_w_{}".format(label): "df: df.sum_w*({})".format(vari)},
                    {"sum_ww_{}".format(label): "df: (df.sum_w*({}))**2".format(vari)},
                ])
    
    monojet_selection = selection + ["MinDPhiJ1234METnoX>0.5"]
    monojetqcd_selection = selection + ["MinDPhiJ1234METnoX<=0.5"]
    singlemuon_selection = selection + ["MinDPhiJ1234METnoX>0.5", "nMuonSelection==1", "MTW>30.", "MTW<125."]
    singlemuonqcd_selection = selection + ["MinDPhiJ1234METnoX<=0.5", "nMuonSelection==1", "MTW>30.", "MTW<125."]
    doublemuon_selection = selection + ["MinDPhiJ1234METnoX>0.5", "nMuonSelection==2", "MLL>71.", "MLL<111.", "LeptonCharge==0."]
    doublemuonqcd_selection = selection + ["MinDPhiJ1234METnoX<=0.5", "nMuonSelection==2", "MLL>71.", "MLL<111.", "LeptonCharge==0."]
    singleelectron_selection = selection + ["MinDPhiJ1234METnoX>0.5", "nElectronSelection==1", "MTW>30.", "MTW<125.", "MET_pt>100."]
    singleelectronqcd_selection = selection + ["MinDPhiJ1234METnoX<=0.5", "nElectronSelection==1", "MTW>30.", "MTW<125.", "MET_pt>100."]
    doubleelectron_selection = selection + ["MinDPhiJ1234METnoX>0.5", "nElectronSelection==2", "MLL>71.", "MLL<111.", "LeptonCharge==0."]
    doubleelectronqcd_selection = selection + ["MinDPhiJ1234METnoX<=0.5", "nElectronSelection==2", "MLL>71.", "MLL<111.", "LeptonCharge==0."]
    singletau_selection = selection + ["MinDPhiJ1234METnoX>0.5", "nTauSelection==1"]
    singlemuonmu_selection = selection + ["MinDPhiJ1234METnoX>0.5", "nMuonSelection==1", "MTW>30.", "MTW<125."]
    singlemuonmuqcd_selection = selection + ["MinDPhiJ1234METnoX<=0.5", "nMuonSelection==1", "MTW>30.", "MTW<125."]
    doublemuonmu_selection = selection + ["MinDPhiJ1234METnoX>0.5", "nMuonSelection==2", "MLL>71.", "MLL<111.", "LeptonCharge==0."]
    doublemuonmuqcd_selection = selection + ["MinDPhiJ1234METnoX<=0.5", "nMuonSelection==2", "MLL>71.", "MLL<111.", "LeptonCharge==0."]
    if not mc:
        monojet_selection.extend(["nMuonVeto==0", "nElectronVeto==0", "nTauVeto==0"])
        monojetqcd_selection.extend(["nMuonVeto==0", "nElectronVeto==0", "nTauVeto==0"])
        singlemuon_selection.extend(["nMuonVeto==1", "nElectronVeto==0", "nTauVeto==0"])
        singlemuonqcd_selection.extend(["nMuonVeto==1", "nElectronVeto==0", "nTauVeto==0"])
        doublemuon_selection.extend(["nMuonVeto==2", "nElectronVeto==0", "nTauVeto==0"])
        doublemuonqcd_selection.extend(["nMuonVeto==2", "nElectronVeto==0", "nTauVeto==0"])
        singleelectron_selection.extend(["nMuonVeto==0", "nElectronVeto==1", "nTauVeto==0"])
        singleelectronqcd_selection.extend(["nMuonVeto==0", "nElectronVeto==1", "nTauVeto==0"])
        doubleelectron_selection.extend(["nMuonVeto==0", "nElectronVeto==2", "nTauVeto==0"])
        doubleelectronqcd_selection.extend(["nMuonVeto==0", "nElectronVeto==2", "nTauVeto==0"])
        singletau_selection.extend(["nMuonVeto==0", "nElectronVeto==0", "nTauVeto==1"])
        singlemuonmu_selection.extend(["nMuonVeto==1", "nElectronVeto==0", "nTauVeto==0"])
        singlemuonmuqcd_selection.extend(["nMuonVeto==1", "nElectronVeto==0", "nTauVeto==0"])
        doublemuonmu_selection.extend(["nMuonVeto==2", "nElectronVeto==0", "nTauVeto==0"])
        doublemuonmuqcd_selection.extend(["nMuonVeto==2", "nElectronVeto==0", "nTauVeto==0"])

    cutflows = {
        "DoubleMuon": {
            "selection_name": "DoubleMuon",
            "selection": "(" + ") & (".join(doublemuon_selection) + ")",
            "weight": "1." if not mc else "df.Weight_DoubleMuon",
            "hists": [h for h in hists_def.keys() if not any(test in h or h.startswith("n") for test in [
                "MTW", "JetFwd", "NearestJet", "Electron", "Tau",
            ])],
        },
        "DoubleElectron": {
            "selection_name": "DoubleElectron",
            "selection": "(" + ") & (".join(doubleelectron_selection) + ")",
            "weight": "1." if not mc else "df.Weight_DoubleElectron",
            "hists": [h for h in hists_def.keys() if not any(test in h or h.startswith("n") for test in [
                "MTW", "JetFwd", "NearestJet", "Muon", "Tau",
            ])],
        },
        "DoubleMuonMu": {
            "selection_name": "DoubleMuonMu",
            "selection": "(" + ") & (".join(doublemuonmu_selection) + ")",
            "weight": "1." if not mc else "df.Weight_DoubleMuonMu",
            "hists": [h for h in hists_def.keys() if not any(test in h or h.startswith("n") for test in [
                "MTW", "JetFwd", "NearestJet", "Electron", "Tau",
            ])],
        },
    }

    met_triggers = [
        "df.HLT_PFMETNoMu90_PFMHTNoMu90_IDTight",
        "df.HLT_PFMETNoMu100_PFMHTNoMu100_IDTight",
        "df.HLT_PFMETNoMu110_PFMHTNoMu110_IDTight",
        "df.HLT_PFMETNoMu120_PFMHTNoMu120_IDTight",
        "(df.HLT_PFMET170_NotCleaned & (df.run<276282))",
        "(df.HLT_PFMET170_BeamHaloCleaned & (df.run<276282))",
        "df.HLT_PFMET170_HBHECleaned",
        "df.HLT_PFMET170_HBHE_BeamHaloCleaned",
        "df.HLT_MET75_IsoTrk50",
    ]
    if mc:
        met_triggers = [
            "df.HLT_PFMETNoMu90_PFMHTNoMu90_IDTight",
            "df.HLT_PFMETNoMu100_PFMHTNoMu100_IDTight",
            "df.HLT_PFMETNoMu110_PFMHTNoMu110_IDTight",
            "df.HLT_PFMETNoMu120_PFMHTNoMu120_IDTight",
            "df.HLT_PFMET170_NotCleaned",
            "df.HLT_PFMET170_BeamHaloCleaned",
            "df.HLT_PFMET170_HBHECleaned",
            "df.HLT_PFMET170_HBHE_BeamHaloCleaned",
            "df.HLT_MET75_IsoTrk50",
        ]
        
    singlemuon_triggers = ["df.HLT_IsoMu24", "df.HLT_IsoTkMu24"]
    singleelectron_triggers = [
        "df.HLT_Ele27_WPTight_Gsf",
        "df.HLT_Ele115_CaloIdVT_GsfTrkIdT",
    ]

    cfg_eval_bins = [
        {"ZCand_px": "df: np.where(df.nMuonSelection==2, df.LeadMuonSelection_pt*np.cos(df.LeadMuonSelection_phi)+df.SecondMuonSelection_pt*np.cos(df.SecondMuonSelection_phi), np.zeros_like(df.nMuonSelection)) + np.where(df.nElectronSelection==2, df.LeadElectronSelection_pt*np.cos(df.LeadElectronSelection_phi)+df.SecondElectronSelection_pt*np.cos(df.SecondElectronSelection_phi), np.zeros_like(df.nMuonSelection))"},
        {"ZCand_py": "df: np.where(df.nMuonSelection==2, df.LeadMuonSelection_pt*np.sin(df.LeadMuonSelection_phi)+df.SecondMuonSelection_pt*np.sin(df.SecondMuonSelection_phi), np.zeros_like(df.nElectronSelection)) + np.where(df.nElectronSelection==2, df.LeadElectronSelection_pt*np.sin(df.LeadElectronSelection_phi)+df.SecondElectronSelection_pt*np.sin(df.SecondElectronSelection_phi), np.zeros_like(df.nElectronSelection))"},
        {"ZCand_pt": "df: np.sqrt(df.ZCand_px**2 + df.ZCand_py**2)"},
        {"ZCand_phi": "df: np.arctan2(df.ZCand_py, df.ZCand_px)"},
        {"METnoX_ptParaZCandCent": "df: df.METnoX_pt*np.cos(df.METnoX_phi-df.ZCand_phi) - df.ZCand_pt"},
        {"METnoX_ptPerpZCandCent": "df: df.METnoX_pt*np.sin(df.METnoX_phi-df.ZCand_phi)"},
        {"ZCand_pt_bin": "df: np.floor(df.ZCand_pt/5.)*5."},
        {"METnoX_ptParaZCandCent_bin": "df: np.floor(df.METnoX_ptParaZCandCent/10.)*10."},
        {"METnoX_ptPerpZCandCent_bin": "df: np.floor(df.METnoX_ptPerpZCandCent/10.)*10."},
    ]
    
    if mc:
        weight = [
            "df.WeightXsLumi",
            "df.WeightQcdEwk",
            "df.WeightPU",
            "df.WeightPreFiring",
            "df.WeightOneMinusPhotonVetoIdLoose",
            "df.WeightOneMinusPhotonVetoPixelSeedVeto",
            "df.WeightOneMinusJetBVetoIdMedium",
        ]
        weight_ele_veto = [
            "df.WeightOneMinusElectronVetoNoSelectionIdIsoVeto",
            "df.WeightOneMinusElectronSelectionIdIsoTight",
            "df.WeightOneMinusElectronVetoReco",
        ]
        weight_ele_sele = [
            "df.WeightElectronSelectionIdIsoTight",
            "df.WeightElectronSelectionReco",
            "df.WeightOneMinusElectronVetoNoSelectionIdIsoVeto",
            "df.WeightOneMinusElectronVetoNoSelectionReco",
        ]
        weight_mu_veto = [
            "df.WeightOneMinusMuonVetoNoSelectionIdLoose",
            "df.WeightOneMinusMuonVetoNoSelectionIsoLoose",
            "df.WeightOneMinusMuonSelectionIdTight",
            "df.WeightOneMinusMuonSelectionIsoTight",
        ]
        weight_mu_sele = [
            "df.WeightMuonSelectionIdTight",
            "df.WeightMuonSelectionIsoTight",
            "df.WeightOneMinusMuonVetoNoSelectionIdLoose",
            "df.WeightOneMinusMuonVetoNoSelectionIsoLoose",
        ]
        weight_tau_veto = ["df.WeightOneMinusTauSelectionIdTight"]
        weight_tau_sele = ["df.WeightTauSelectionIdTight"]
        weight_met_trig = ["df.WeightMETTrig",]
        weight_singleele_trig = ["df.WeightSingleElectronTrig"]
        weight_singlemu_trig = ["df.WeightSingleMuonTrig"]
        
        
        
        cfg_eval_bins.extend([
            {"parent": "df: np.where( df.parent.isin(['WJetsToLNu', 'DYJetsToLL']), np.where( df.parent=='WJetsToLNu', np.where( df.LeptonIsElectron, np.full_like(df.parent, 'WJetsToENu'), np.where( df.LeptonIsMuon, np.full_like(df.parent, 'WJetsToMuNu'), np.where( df.LeptonIsTau, np.full_like(df.parent, 'WJetsToTauNu'), np.full_like(df.parent, 'WJetsToLNu'),),),), np.where( df.parent=='DYJetsToLL', np.where( df.LeptonIsElectron, np.full_like(df.parent, 'DYJetsToEE'), np.where( df.LeptonIsMuon, np.full_like(df.parent, 'DYJetsToMuMu'), np.where( df.LeptonIsTau, np.where( df.nGenTauL==0, np.full_like(df.parent, 'DYJetsToTauHTauH'), np.where( df.nGenTauL==1, np.full_like(df.parent, 'DYJetsToTauHTauL'), np.where( df.nGenTauL==2, np.full_like(df.parent, 'DYJetsToTauLTauL'), np.full_like(df.parent, 'DYJetsToTauTau'),),),), np.full_like(df.parent, 'DYJetsToLL'),),),), df.parent,),), df.parent,)"},
            {"Weight_DoubleMuon": "df: (" + ")*(".join(weight+weight_ele_veto+weight_mu_sele+weight_tau_veto+weight_met_trig)+")"},
            {"Weight_DoubleElectron": "df: (" + ")*(".join(weight+weight_ele_sele+weight_mu_veto+weight_tau_veto+weight_singleele_trig)+")"},
            {"Weight_DoubleMuonMu": "df: (" + ")*(".join(weight+weight_ele_veto+weight_mu_sele+weight_tau_veto+weight_singlemu_trig)+")"},
        ])

    cfg = {
        "query": {
            "groupby": ["table", "varname0", "varname1", "selection", "parent", "binvar0", "binvar1"],
            "tables": tables,
            "aliases": {},
            "eval": cfg_eval_bins,
            "cutflows": cutflows,
            "hists": hists_def,
        },
        "files": sorted(p for p in glob.glob(filepath)),
    }
    
    return cfg

In [17]:
hists = [
    ("ZCand_pt", "METnoX_ptParaZCandCent"),
    ("ZCand_pt", "METnoX_ptPerpZCandCent"),
]

systs = [
    ("d1kqcdUp", "df.WeightQcdEwk_d1kqcdUp/df.WeightQcdEwk"),
    ("d1kqcdDown", "df.WeightQcdEwk_d1kqcdDown/df.WeightQcdEwk"),
    ("d2kqcdUp", "df.WeightQcdEwk_d2kqcdUp/df.WeightQcdEwk"),
    ("d2kqcdDown", "df.WeightQcdEwk_d2kqcdDown/df.WeightQcdEwk"),
    ("d3kqcdUp", "df.WeightQcdEwk_d3kqcdUp/df.WeightQcdEwk"),
    ("d3kqcdDown", "df.WeightQcdEwk_d3kqcdDown/df.WeightQcdEwk"),
    ("d1kewUp", "df.WeightQcdEwk_d1kewUp/df.WeightQcdEwk"),
    ("d1kewDown", "df.WeightQcdEwk_d1kewDown/df.WeightQcdEwk"),
    ("d2kewzUp", "df.WeightQcdEwk_d2kewzUp/df.WeightQcdEwk"),
    ("d2kewzDown", "df.WeightQcdEwk_d2kewzDown/df.WeightQcdEwk"),
    ("d2kewwUp", "df.WeightQcdEwk_d2kewwUp/df.WeightQcdEwk"),
    ("d2kewwDown", "df.WeightQcdEwk_d2kewwDown/df.WeightQcdEwk"),
    ("d3kewzUp", "df.WeightQcdEwk_d3kewzUp/df.WeightQcdEwk"),
    ("d3kewzDown", "df.WeightQcdEwk_d3kewzDown/df.WeightQcdEwk"),
    ("d3kewwUp", "df.WeightQcdEwk_d3kewwUp/df.WeightQcdEwk"),
    ("d3kewwDown", "df.WeightQcdEwk_d3kewwDown/df.WeightQcdEwk"),
    ("dkmixUp", "df.WeightQcdEwk_dkmixUp/df.WeightQcdEwk"),
    ("dkmixDown", "df.WeightQcdEwk_dkmixDown/df.WeightQcdEwk"),
    ("pileupUp", "df.WeightPU_pileupUp/df.WeightPU"),
    ("pileupDown", "df.WeightPU_pileupDown/df.WeightPU"), 
    ("prefiringUp", "df.WeightPreFiring_prefiringUp/df.WeightPreFiring"),
    ("prefiringDown", "df.WeightPreFiring_prefiringDown/df.WeightPreFiring"),
    ("muonTrigUp", "np.where(df.selection.str.contains('MuonMu'), df.WeightSingleMuonTrig_muonTrigUp/df.WeightSingleMuonTrig, np.ones_like(df.WeightSingleMuonTrig))"),
    ("muonTrigDown", "np.where(df.selection.str.contains('MuonMu'), df.WeightSingleMuonTrig_muonTrigDown/df.WeightSingleMuonTrig, np.ones_like(df.WeightSingleMuonTrig))"),
    ("muonIdLooseStatUp", 'df.WeightOneMinusMuonVetoNoSelectionIdLoose_muonIdLooseStatUp/df.WeightOneMinusMuonVetoNoSelectionIdLoose'),
    ("muonIdLooseStatDown", 'df.WeightOneMinusMuonVetoNoSelectionIdLoose_muonIdLooseStatDown/df.WeightOneMinusMuonVetoNoSelectionIdLoose'),
    ("muonIdLooseSystUp", 'df.WeightOneMinusMuonVetoNoSelectionIdLoose_muonIdLooseSystUp/df.WeightOneMinusMuonVetoNoSelectionIdLoose'),
    ("muonIdLooseSystDown", 'df.WeightOneMinusMuonVetoNoSelectionIdLoose_muonIdLooseSystDown/df.WeightOneMinusMuonVetoNoSelectionIdLoose'),
    ("muonIsoLooseStatUp", 'df.WeightOneMinusMuonVetoNoSelectionIsoLoose_muonIsoLooseStatUp/df.WeightOneMinusMuonVetoNoSelectionIsoLoose'),
    ("muonIsoLooseStatDown", 'df.WeightOneMinusMuonVetoNoSelectionIsoLoose_muonIsoLooseStatDown/df.WeightOneMinusMuonVetoNoSelectionIsoLoose'),
    ("muonIsoLooseSystUp", 'df.WeightOneMinusMuonVetoNoSelectionIsoLoose_muonIsoLooseSystUp/df.WeightOneMinusMuonVetoNoSelectionIsoLoose'),
    ("muonIsoLooseSystDown", 'df.WeightOneMinusMuonVetoNoSelectionIsoLoose_muonIsoLooseSystDown/df.WeightOneMinusMuonVetoNoSelectionIsoLoose'),
    ("muonIdTightStatUp", "np.where(df.selection.str.contains('Muon'), df.WeightMuonSelectionIdTight_muonIdTightStatUp/df.WeightMuonSelectionIdTight, df.WeightOneMinusMuonSelectionIdTight_muonIdTightStatUp/df.WeightOneMinusMuonSelectionIdTight)"),
    ("muonIdTightStatDown", "np.where(df.selection.str.contains('Muon'), df.WeightMuonSelectionIdTight_muonIdTightStatDown/df.WeightMuonSelectionIdTight, df.WeightOneMinusMuonSelectionIdTight_muonIdTightStatDown/df.WeightOneMinusMuonSelectionIdTight)"),
    ("muonIdTightSystUp", "np.where(df.selection.str.contains('Muon'), df.WeightMuonSelectionIdTight_muonIdTightSystUp/df.WeightMuonSelectionIdTight, df.WeightOneMinusMuonSelectionIdTight_muonIdTightSystUp/df.WeightOneMinusMuonSelectionIdTight)"),
    ("muonIdTightSystDown", "np.where(df.selection.str.contains('Muon'), df.WeightMuonSelectionIdTight_muonIdTightSystDown/df.WeightMuonSelectionIdTight, df.WeightOneMinusMuonSelectionIdTight_muonIdTightSystDown/df.WeightOneMinusMuonSelectionIdTight)"),
    ("muonIsoTightStatUp", "np.where(df.selection.str.contains('Muon'), df.WeightMuonSelectionIsoTight_muonIsoTightStatUp/df.WeightMuonSelectionIsoTight, df.WeightOneMinusMuonSelectionIsoTight_muonIsoTightStatUp/df.WeightOneMinusMuonSelectionIsoTight)"),
    ("muonIsoTightStatDown", "np.where(df.selection.str.contains('Muon'), df.WeightMuonSelectionIsoTight_muonIsoTightStatDown/df.WeightMuonSelectionIsoTight, df.WeightOneMinusMuonSelectionIsoTight_muonIsoTightStatDown/df.WeightOneMinusMuonSelectionIsoTight)"),
    ("muonIsoTightSystUp", "np.where(df.selection.str.contains('Muon'), df.WeightMuonSelectionIsoTight_muonIsoTightSystUp/df.WeightMuonSelectionIsoTight, df.WeightOneMinusMuonSelectionIsoTight_muonIsoTightSystUp/df.WeightOneMinusMuonSelectionIsoTight)"),
    ("muonIsoTightSystDown", "np.where(df.selection.str.contains('Muon'), df.WeightMuonSelectionIsoTight_muonIsoTightSystDown/df.WeightMuonSelectionIsoTight, df.WeightOneMinusMuonSelectionIsoTight_muonIsoTightSystDown/df.WeightOneMinusMuonSelectionIsoTight)"),
    ("eleTrigUp", "np.where(df.selection.str.contains('Electron'), df.WeightSingleElectronTrig_eleTrigUp/df.WeightSingleElectronTrig, np.ones_like(df.WeightSingleElectronTrig))"),
    ("eleTrigDown", "np.where(df.selection.str.contains('Electron'), df.WeightSingleElectronTrig_eleTrigDown/df.WeightSingleElectronTrig, np.ones_like(df.WeightSingleElectronTrig))"),
    ("eleIdIsoVetoUp", 'df.WeightOneMinusElectronVetoNoSelectionIdIsoVeto_eleIdIsoVetoUp/df.WeightOneMinusElectronVetoNoSelectionIdIsoVeto'),
    ("eleIdIsoVetoDown", 'df.WeightOneMinusElectronVetoNoSelectionIdIsoVeto_eleIdIsoVetoDown/df.WeightOneMinusElectronVetoNoSelectionIdIsoVeto'),
    ("eleIdIsoTightUp", "np.where(df.selection.str.contains('Electron'), df.WeightElectronSelectionIdIsoTight_eleIdIsoTightUp/df.WeightElectronSelectionIdIsoTight, df.WeightOneMinusElectronSelectionIdIsoTight_eleIdIsoTightUp/df.WeightOneMinusElectronSelectionIdIsoTight)"),
    ("eleIdIsoTightDown", "np.where(df.selection.str.contains('Electron'), df.WeightElectronSelectionIdIsoTight_eleIdIsoTightDown/df.WeightElectronSelectionIdIsoTight, df.WeightOneMinusElectronSelectionIdIsoTight_eleIdIsoTightDown/df.WeightOneMinusElectronSelectionIdIsoTight)"),
    ("eleRecoUp", "np.where(df.selection.str.contains('Electron'), df.WeightElectronSelectionReco_eleRecoUp*df.WeightOneMinusElectronVetoNoSelectionReco_eleRecoUp/(df.WeightElectronSelectionReco*df.WeightOneMinusElectronVetoNoSelectionReco), df.WeightOneMinusElectronVetoReco_eleRecoUp/df.WeightOneMinusElectronVetoReco)"),
    ("eleRecoDown", "np.where(df.selection.str.contains('Electron'), df.WeightElectronSelectionReco_eleRecoDown*df.WeightOneMinusElectronVetoNoSelectionReco_eleRecoDown/(df.WeightElectronSelectionReco*df.WeightOneMinusElectronVetoNoSelectionReco), df.WeightOneMinusElectronVetoReco_eleRecoDown/df.WeightOneMinusElectronVetoReco)"),
    ("photonIdLooseUp", "df.WeightOneMinusPhotonVetoIdLoose_photonIdLooseUp/df.WeightOneMinusPhotonVetoIdLoose"),
    ("photonIdLooseDown", "df.WeightOneMinusPhotonVetoIdLoose_photonIdLooseDown/df.WeightOneMinusPhotonVetoIdLoose"),
    ("photonPixelSeedVetoUp", "df.WeightOneMinusPhotonVetoPixelSeedVeto_photonPixelSeedVetoUp/df.WeightOneMinusPhotonVetoPixelSeedVeto"),
    ("photonPixelSeedVetoDown", "df.WeightOneMinusPhotonVetoPixelSeedVeto_photonPixelSeedVetoDown/df.WeightOneMinusPhotonVetoPixelSeedVeto"),
    ("tauIdVLooseUp", "df.WeightOneMinusTauVetoNoSelectionIdVLoose_tauIdVLooseUp/df.WeightOneMinusTauVetoNoSelectionIdVLoose"),
    ("tauIdVLooseDown", "df.WeightOneMinusTauVetoNoSelectionIdVLoose_tauIdVLooseDown/df.WeightOneMinusTauVetoNoSelectionIdVLoose"),
    ("tauIdTightUp", "np.where(df.selection.str.contains('Tau'), df.WeightTauSelectionIdTight_tauIdTightUp/df.WeightTauSelectionIdTight, df.WeightOneMinusTauSelectionIdTight_tauIdTightUp/df.WeightOneMinusTauSelectionIdTight)"),
    ("tauIdTightDown", "np.where(df.selection.str.contains('Tau'), df.WeightTauSelectionIdTight_tauIdTightDown/df.WeightTauSelectionIdTight, df.WeightOneMinusTauSelectionIdTight_tauIdTightDown/df.WeightOneMinusTauSelectionIdTight)"),
    ("btagSFUp", 'df.WeightOneMinusJetBVetoIdMedium_btagSFUp/df.WeightOneMinusJetBVetoIdMedium'),
    ("btagSFDown", 'df.WeightOneMinusJetBVetoIdMedium_btagSFDown/df.WeightOneMinusJetBVetoIdMedium'),
    ("metTrigMuonMultiplicitySystUp", "np.where(~(df.selection.str.contains('Electron') | df.selection.str.contains('MuonMu')), df.WeightMETTrig_muonMultiplicitySystUp/df.WeightMETTrig, np.ones_like(df.WeightMETTrig))"),
    ("metTrigMuonMultiplicitySystDown", "np.where(~(df.selection.str.contains('Electron') | df.selection.str.contains('MuonMu')), df.WeightMETTrig_muonMultiplicitySystDown/df.WeightMETTrig, np.ones_like(df.WeightMETTrig))"),
    ("metTrigReferenceTriggerSystUp", "np.where(~(df.selection.str.contains('Electron') | df.selection.str.contains('MuonMu')), df.WeightMETTrig_referenceTriggerSystUp/df.WeightMETTrig, np.ones_like(df.WeightMETTrig))"),
    ("metTrigReferenceTriggerSystDown", "np.where(~(df.selection.str.contains('Electron') | df.selection.str.contains('MuonMu')), df.WeightMETTrig_referenceTriggerSystDown/df.WeightMETTrig, np.ones_like(df.WeightMETTrig))"),
    ("metTrigRegionSystUp", "np.where(~(df.selection.str.contains('Electron') | df.selection.str.contains('MuonMu')), df.WeightMETTrig_regionSystUp/df.WeightMETTrig, np.ones_like(df.WeightMETTrig))"),
    ("metTrigRegionSystDown", "np.where(~(df.selection.str.contains('Electron') | df.selection.str.contains('MuonMu')), df.WeightMETTrig_regionSystDown/df.WeightMETTrig, np.ones_like(df.WeightMETTrig))"),
] + [
    ("lheScaleWeight{}".format(idx), "np.where(~(df.parent.str.contains('ZJetsTo') | df.parent.str.contains('WJetsTo') | df.parent.str.contains('DYJetsTo') | df.parent.str.contains('GStarJetsTo')), df.LHEScaleWeight{}, np.ones_like(df.LHEScaleWeight0))".format(idx))
    for idx in range(9)
] + [
    ("lhePdfWeight{}".format(idx), "df.LHEPdfWeight{}".format(idx))
    for idx in range(104)
]

cfg_data = generate_yaml(
    hists,
    ["IsCertified", "Flag_goodVertices", "Flag_globalSuperTightHalo2016Filter", "Flag_HBHENoiseFilter", "Flag_HBHENoiseIsoFilter", "Flag_EcalDeadCellTriggerPrimitiveFilter", "Flag_BadPFMuonFilter", "Flag_eeBadScFilter", "MET_dCaloMET<0.6", "nJetSelection>0", "nJetSelection==nJetVeto", "LeadJetSelection_chHEF>0.1", "LeadJetSelection_neHEF<0.8", "LeadJetSelection_pt>200.", "nPhotonVeto==0", "nBJetVeto==0"],
    "/vols/cms/sdb15/Analysis/ZinvWidth/databases/full/2020/02_Feb/10_SingleTable_FixObjectWeights/Data/*.h5",
    mc=False,
)
with open("configs/data.yaml", "w") as f:
    yaml.dump(cfg_data, f, indent=4)
    
cfg_mc = generate_yaml(
    hists,
    ["(parent!='EWKV2Jets' | nGenBosonSelection==1)", "Flag_goodVertices", "Flag_globalSuperTightHalo2016Filter", "Flag_HBHENoiseFilter", "Flag_HBHENoiseIsoFilter", "Flag_EcalDeadCellTriggerPrimitiveFilter", "Flag_BadPFMuonFilter", "MET_dCaloMET<0.6", "nJetSelection>0", "nJetSelection==nJetVeto", "LeadJetSelection_chHEF>0.1", "LeadJetSelection_neHEF<0.8", "LeadJetSelection_pt>200."],
    "/vols/cms/sdb15/Analysis/ZinvWidth/databases/full/2020/02_Feb/10_SingleTable_FixObjectWeights/MC/*.h5",
    systs=systs,
    mc=True,
)
with open("configs/mc.yaml", "w") as f:
    yaml.dump(cfg_mc, f, indent=4)

cfg_mc_jec = generate_yaml(
    hists,
    ["(parent!='EWKV2Jets' | nGenBosonSelection==1)", "Flag_goodVertices", "Flag_globalSuperTightHalo2016Filter", "Flag_HBHENoiseFilter", "Flag_HBHENoiseIsoFilter", "Flag_EcalDeadCellTriggerPrimitiveFilter", "Flag_BadPFMuonFilter", "MET_dCaloMET<0.6", "nJetSelection>0", "nJetSelection==nJetVeto", "LeadJetSelection_chHEF>0.1", "LeadJetSelection_neHEF<0.8", "LeadJetSelection_pt>200."],
    "/vols/cms/sdb15/Analysis/ZinvWidth/databases/full/2020/02_Feb/10_SingleTable_FixObjectWeights/MC_JEC/*.h5",
    tables={"jesTotalUp": "Events_jesTotalup", "jesTotalUp": "Events_jesTotaldown", "jerSFUp": "Events_jerSFup", "jerSFUp": "Events_jerSFdown", "unclsutUp": "Events_unclustup", "unclsutUp": "Events_unclustdown"},
    mc=True,
)
with open("configs/mc_jec.yaml", "w") as f:
    yaml.dump(cfg_mc_jec, f, indent=4)
    
cfg_mc_lep = generate_yaml(
    hists,
    ["(parent!='EWKV2Jets' | nGenBosonSelection==1)", "Flag_goodVertices", "Flag_globalSuperTightHalo2016Filter", "Flag_HBHENoiseFilter", "Flag_HBHENoiseIsoFilter", "Flag_EcalDeadCellTriggerPrimitiveFilter", "Flag_BadPFMuonFilter", "MET_dCaloMET<0.6", "nJetSelection>0", "nJetSelection==nJetVeto", "LeadJetSelection_chHEF>0.1", "LeadJetSelection_neHEF<0.8", "LeadJetSelection_pt>200."],
    "/vols/cms/sdb15/Analysis/ZinvWidth/databases/full/2020/02_Feb/10_SingleTable_FixObjectWeights/MC_LEP/*.h5",
    tables={"eleEnergyScaleUp": "Events_eleEnergyScaleup", "eleEnergyScaleDown": "Events_eleEnergyScaledown", "muonPtScaleUp": "Events_muonPtScaleup", "muonPtScaleDown": "Events_muonPtScaledown", "photonEnergyScaleUp": "Events_photonEnergyScaleup", "photonEnergyScaleDown": "Events_photonEnergyScaledown", "tauPtScaleUp": "Events_tauPtScaleup", "tauPtScaleDown": "Events_tauPtScaledown"},
    mc=True,
)
with open("configs/mc_lep.yaml", "w") as f:
    yaml.dump(cfg_mc_lep, f, indent=4)

In [18]:
!~/Scripts/batch/QSTAT.py


 queue   |   free |   used |   total
---------+--------+--------+---------
 hep.q   |     15 |    425 |     440
 gpu.q   |      7 |      0 |       7
 fw.q    |      4 |      0 |       4

 user        |     hep.q |           |
             |   running |   pending | duration
-------------+-----------+-----------+-----------------
 Eshwen      |       190 |       976 | 0 days 00:02:41
 Giacomo     |       128 |      2325 | 0 days 00:48:41
 Prince Albz |        55 |      3692 | 0 days 01:55:41
 Danny       |        42 |      2079 | 0 days 02:58:56
 The GOAT    |         8 |       195 | 0 days 02:40:11



In [19]:
#zdb.modules.multi_analyse(
#    ["configs/data.yaml", "configs/mc.yaml", "configs/mc_jec.yaml", "configs/mc_lep.yaml"],
#    outputs=[
#        "met_scale_res.h5:DataAggEvents", "met_scale_res.h5:MCAggEvents",
#        "met_scale_res.h5:MCAggEvents_jec", "met_scale_res.h5:MCAggEvents_lep",
#    ],
#    mode='sge',
#    ncores=-1,
#    batch_opts="-q hep.q -l h_rt=3:0:0 -l h_vmem=12G",
#    chunksize=400_000,
#    merge_opts={"mode": "sge", "ncores": 25, "batch_opts": "-q hep.q"},
#)